# A simple quantum circuit compiler

by Theodor Lundberg

While one may construct quantum circuits with a range of gates, the hardware of a quantum processor will often limit which gates can be implemented--this subset of gates is called the natural gate set of the quantum processor. When feeding a quantum circuit to a certain type of hardware, it must therefore first be compiled such that the it only contains gates belonging to natural gate set. In this notebook, I introduce a very basic compiler which is capable of taking a qiskit.QuantumCircuit constituted by arbitrary simple 1- and 2-qubit gates and compiling it to a QuantumCircuit with only the CZ, RX and RZ gates.

We start by importing the relevant modules.

In [1]:
import numpy as np
from math import pi
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
#from qiskit.circuit.library.standard_gates import YGate, RYGate, RXGate, RZGate
from qiskit.converters import circuit_to_dag

# Compiler program

### Decomposing gates to restricted gate set

With help from Chapter 4 of Nielsen and Chuang, the simple gates I, H, X, Y, Z, RX, RY, RZ, CNOT and CZ can be decomposed into the restricted gate set composed of RX, RZ and CZ:
* I = RX(0) = RZ(0)
* H = RX($\pi$/2)-RZ($\pi$/2)-RX($\pi$/2)
* X = RX($\pi$)
* Y = RZ(-$\pi$/2)-RX($\pi$)-RZ($\pi$/2)
* Z = RZ($\pi$)
* RX($\theta$) = RX($\theta$)
* RY($\theta$) = RZ(-$\pi$/2)-RX($\theta$)-RZ($\pi$/2)
* RZ($\theta$) = RZ($\theta$)
* CZ(control, target) = CZ(control, target)
* CNOT(control, target) = H(target)-CZ(control, target)-H(target) = RX($\pi$/2)-RZ($\pi$/2)-RX($\pi$/2)-CZ(control, target)-RX($\pi$/2)-RZ($\pi$/2)-RX($\pi$/2)

Based on this, we can write a simple compiler that takes a qiskit QuantumCircuit object with various simple gates and returns an equivalent circuit with gates only from the restricted gate set:

In [2]:
def compile_to_RX_RX_CZ(circ):
    '''
    args
    ----
    circ : qiskit.QuantumCircuit object containing gates from the set {I, H, X, Y, Z, RX, RY, RZ, CNOT, CZ}.
    
    returns
    -------
    comp_circ : qiskit.QuantumCircuit object containing gates from the set {RX, RZ, CZ}.
    '''
    # initialise the circuit for the compiled version of the input circuit
    comp_circ = QuantumCircuit(circ.qregs[0],circ.cregs[0])
    
    # loop through the gates in the circuit
    for gate in circ:
        name = gate[0].name
        qubits = gate[1]
        clbits = gate[2]
        params = gate[0].params
        
        
        # decompose gates and add to compilation circuit
        if name == 'cz':
            comp_circ.cz(qubits[0],qubits[1])
        
        elif name == 'cx': #cnot
            comp_circ.rz(pi/2,qubits[1])
            comp_circ.rx(pi/2,qubits[1])
            comp_circ.rz(pi/2,qubits[1])
            comp_circ.cz(qubits[0],qubits[1])
            comp_circ.rz(pi/2,qubits[1])
            comp_circ.rx(pi/2,qubits[1])
            comp_circ.rz(pi/2,qubits[1])
            
        elif name == 'rx':
            comp_circ.rx(params[0],qubits)
            
        elif name == 'rz':
            comp_circ.rz(params[0],qubits)
        
        elif name == 'ry':
            comp_circ.rz(-pi/2,qubits)
            comp_circ.rx(params[0],qubits)
            comp_circ.rz(pi/2,qubits)
            
        elif name == 'measure':
            comp_circ.measure(qubits,clbits)
            
        elif name == 'h':
            comp_circ.rz(pi/2,qubits)
            comp_circ.rx(pi/2,qubits)
            comp_circ.rz(pi/2,qubits)
        
        elif name == 'id':
            pass
        
        elif name == 'x':
            comp_circ.rx(pi,qubits)
            
        elif name == 'z':
            comp_circ.rz(pi,qubits)
        
        elif name == 'y':
            comp_circ.rz(-pi/2,qubits)
            comp_circ.rx(pi,qubits)
            comp_circ.rz(pi/2,qubits)

    return comp_circ

# Compilation of a basic quantum circuit

Here, we shall consider a basic example circuit containing the following gates: I, H, X, Y, Z, RX, RY, RZ, CNOT and CZ.

In [3]:
q = QuantumRegister(3, 'q')
c = ClassicalRegister(3, 'c')
circ = QuantumCircuit(q, c)

circ.rx(pi/3,0)
circ.h(2)
circ.h(1)
circ.i(0)
circ.y(1)
circ.cnot(1,0)
circ.x(0)
circ.z(2)
circ.rx(pi/2,0)
circ.ry(pi/3,1)
circ.rz(pi/4,2)
circ.cz(1,2)
circ.draw()

┌──────────┐┌───┐   ┌───┐       ┌───┐    ┌──────────┐
q_0: ┤ RX(pi/3) ├┤ I ├───┤ X ├───────┤ X ├────┤ RX(pi/2) ├
     └──┬───┬───┘├───┤   └─┬─┘    ┌──┴───┴───┐└──────────┘
q_1: ───┤ H ├────┤ Y ├─────■──────┤ RY(pi/3) ├─────■──────
        ├───┤    ├───┤┌──────────┐└──────────┘     │      
q_2: ───┤ H ├────┤ Z ├┤ RZ(pi/4) ├─────────────────■──────
        └───┘    └───┘└──────────┘                        
c: 3/═════════════════════════════════════════════════════

Using the compilation programme, the circuit is compiled to the restricted gate set:

In [4]:
compiled_circ = compile_to_RX_RX_CZ(circ)
compiled_circ.draw()

┌──────────┐┌──────────┐┌──────────┐ ┌──────────┐                        »
q_0: ┤ RX(pi/3) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├─┤ RZ(pi/2) ├────────────────────────»
     ├──────────┤├──────────┤├──────────┤┌┴──────────┤ ┌────────┐ ┌──────────┐»
q_1: ┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(-pi/2) ├─┤ RX(pi) ├─┤ RZ(pi/2) ├»
     ├──────────┤├──────────┤├──────────┤└─┬────────┬┘┌┴────────┴┐└──────────┘»
q_2: ┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi/2) ├──┤ RZ(pi) ├─┤ RZ(pi/4) ├────────────»
     └──────────┘└──────────┘└──────────┘  └────────┘ └──────────┘            »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«         ┌──────────┐┌──────────┐┌──────────┐┌────────┐┌──────────┐
«q_0: ─■──┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RX(pi) ├┤ RX(pi/2) ├
«      │ ┌┴──────────┤├──────────┤├──────────┤└────────┘└──────────┘
«q_1: ─■─┤ RZ(-pi/2) ├┤ RX(pi/3) ├┤ RZ(pi/2) ├────■─────────────────
«        └───────────┘└──────────┘└──────────┘    │                 
«q_2: ────────────────────────────────────────────■─────────────────
«                                                                   
«c: 3/══════════════════════════════════════════════════════════════
«

# Characterisation of the compilation
First we need to check whether the compiled circuit is equivalent to the original circuit. It is important to note that a difference in global phase between the two circuits is allowed as global phase does not affect measurement outcome. We therefore introduce a helper function that can check whether two unitary matrixes are equivalent:

In [5]:
def dagger(M):
    '''
    args
    ----
    M : numpy matrix to be conjugated a transposed
    
    returns
    -------
    M_dag : numpy matrix; M dagger
    '''
    M_dag = np.conj(M).T
    return M_dag

def globalPhaseEquiv(M1, M2):
    '''
    args
    ----
    M1 : 2D numpy matrix
    M2 : 2D numpy matrix
    
    returns
    -------
    equiv : bool indicating whether M1 and M2 are equivalent up to a global phase
    '''
    P = M1.dot(dagger(M2))
    I = np.diag(np.ones(M1.shape[0]))
    if P[0][0] == 0:
        equiv = False
    else: 
        equiv = np.allclose(I,P/P[0][0])
    return equiv

Using IBMs Aer Unitary Simulator we can obtain the unitaries that describe the original and compiled circuits and use the function globalPhaseEquiv to check if the two are equivalent:

In [6]:
backend = Aer.get_backend('unitary_simulator')

sim = execute(circ, backend)
result = sim.result()
U_circ = result.get_unitary(circ, decimals=3)

sim_compiled = execute(compiled_circ, backend)
result_compiled = sim_compiled.result()
U_compiled_circ = result_compiled.get_unitary(compiled_circ, decimals=3)

print(globalPhaseEquiv(U_circ, U_compiled_circ))

True


Fortunately the two circuits are equivalent. Now we can check the overhead that has been introduced through the compilation. While it may be relevant to factor in gate fidelity and gate operation times when calculating the overhead for a physical system, here we shall simply assume the all gates have equal fidelities and operation times. In this case, the overhead is simply given by the number of gate operations in the circuit, which for the original and compiled circuits are:

In [7]:
print('Number of gates in original circuit: {0}'.format(circ.size()))
print('Number of gates in compiled circuit: {0}'.format(compiled_circ.size()))
print('\n ----- COMPILATION OVERHEAD ----- \n Additional gates: {0} \n Percentage increase: {1}%'.format(compiled_circ.size()-circ.size(),round((compiled_circ.size()-circ.size())/circ.size()*100)))

Number of gates in original circuit: 12
Number of gates in compiled circuit: 25

 ----- COMPILATION OVERHEAD ----- 
 Additional gates: 13 
 Percentage increase: 108%


# Optimising the compilation

In the compiler program above, I have not introduced any optimisation whatsoever. One straightforward step that can be taken to optimise the compiled circuit and reduce the overhead, is to combine multiple rotation gates about the same axis into one gate and to remove gates that are simply equivalent to the identity operator:

In [8]:
def basic_optimiser(circ):
    '''
    args
    ----
    circ : qiskit.QuantumCircuit object containing gates from the set {RX, RZ, CZ}.
    
    returns
    -------
    opt_circ : qiskit.QuantumCircuit object containing gates from the set {RX, RZ, CZ}.
    '''
    # initialise the circuit for the compiled version of the input circuit
    opt_circ = QuantumCircuit(circ.qregs[0],circ.cregs[0])
    
    # convert circuit to directed acyclic graph, such that we can step through each gate in
    # the order they operate on the qubits
    dag = circuit_to_dag(circ)
    
    # step through each gate in order of execution
    for i, gate in enumerate(dag.topological_op_nodes()):
        # loop initilisation
        if i == 0: 
            prev_gate = gate
            angle = 0
        
        # if cz gate, add previous rx/rz gate to circuit with aggregated angle, reset angle and then add cz
        if gate.name == 'cz':
            if prev_gate.name == 'rx' and not angle%(2*pi) == 0:
                opt_circ.rx(angle,prev_gate.qargs[0])
            elif prev_gate.name == 'rz' and not angle%(2*pi) == 0:
                opt_circ.rz(angle,prev_gate.qargs[0])
            angle = 0
            opt_circ.cz(gate.qargs[0],gate.qargs[1])
        
        # if gate type is equal to last gate and operate on same qubit, increase aggregate angle
        elif gate.name == prev_gate.name and gate.qargs[0] == prev_gate.qargs[0]:
            angle += gate.op.params[0]
        
        # if gate type is changed and aggregate angle is not a multiple of 2pi, add rz/rx and reset angle
        else:
            if prev_gate.name == 'rx' and not angle%(2*pi) == 0:
                opt_circ.rx(angle,prev_gate.qargs[0])
            elif prev_gate.name == 'rz' and not angle%(2*pi) == 0:
                opt_circ.rz(angle,prev_gate.qargs[0])
            angle = gate.op.params[0]
        
        # redefine previous gate before next iteration of loop
        prev_gate = gate
    
    # add last rotation gate to circuit if there is unused angle
    if not angle == 0:
        if prev_gate.name == 'rx':
            opt_circ.rx(angle,prev_gate.qargs[0])
        elif prev_gate.name == 'rz':
            opt_circ.rz(angle,prev_gate.qargs[0])
        
    return opt_circ

Using this optimiser we get the following circuit, which is equivalent to the original circuit and has a reduced overhead compared to the compiled, unoptimised circuit:

In [9]:
opt_circ = basic_optimiser(compiled_circ)
opt_circ.draw()

┌──────────┐┌──────────┐ ┌──────────┐┌──────────┐    ┌──────────┐»
q_0: ┤ RX(pi/3) ├┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├─■──┤ RZ(pi/2) ├»
     ├──────────┤├──────────┤ └┬────────┬┘├──────────┤ │ ┌┴──────────┤»
q_1: ┤ RZ(pi/2) ├┤ RX(pi/2) ├──┤ RX(pi) ├─┤ RZ(pi/2) ├─■─┤ RZ(-pi/2) ├»
     ├──────────┤├──────────┤┌─┴────────┴┐└──────────┘   └───────────┘»
q_2: ┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(7pi/4) ├────────────────────────────»
     └──────────┘└──────────┘└───────────┘                            »
c: 3/═════════════════════════════════════════════════════════════════»
                                                                      »
«     ┌──────────┐┌──────────┐┌───────────┐
«q_0: ┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RX(3pi/2) ├
«     ├──────────┤├──────────┤└───────────┘
«q_1: ┤ RX(pi/3) ├┤ RZ(pi/2) ├──────■──────
«     └──────────┘└──────────┘      │      
«q_2: ──────────────────────────────■──────
«                                          
«c: 3/═════════════════════════════════════
«

In [10]:
sim_opt = execute(opt_circ, backend)
result_opt = sim_opt.result()
U_opt_circ = result_opt.get_unitary(opt_circ, decimals=3)

print(globalPhaseEquiv(U_circ, U_opt_circ))

True


In [11]:
print('Number of gates in original circuit: {0}'.format(circ.size()))
print('Number of gates in compiled and optimised circuit: {0}'.format(opt_circ.size()))
print('\n ----- OPTIMISED COMPILATION OVERHEAD ----- \n Additional gates: {0} \n Percentage increase: {1}%'.format(opt_circ.size()-circ.size(),round((opt_circ.size()-circ.size())/circ.size()*100)))

Number of gates in original circuit: 12
Number of gates in compiled and optimised circuit: 20

 ----- OPTIMISED COMPILATION OVERHEAD ----- 
 Additional gates: 8 
 Percentage increase: 67%


Note that the overhead has been significantly reduced in comparison to the unoptimised, compiled circuit. For further optimisation of the circuit, one could search for and remove unnessary global phases within the circuit.